In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
file_path2 = 'HLA_concordance_cleaned2.csv'
HLA_concordance = pd.read_csv(file_path2)

In [ ]:
def check_combinations(row):
    # Initialize the new columns
    row['hibag_DQ2.5'], row['hibag_DQ2.2'], row['hibag_DQ7.5'], row['hibag_DQ8.1'] = 'negative', 'negative', 'negative', 'negative'
    
    # Condition for hibag_DQ2.5
    if '5:01:00' in [row['hibag.DQA1.1'], row['hibag.DQA1.2']] and '2:01:00' in [row['hibag.DQB1.1'], row['hibag.DQB1.2']]:
        row['hibag_DQ2.5'] = 'positive'
        
    # Condition for hibag_DQ2.2
    if '2:02:00' in [row['hibag.DQB1.1'], row['hibag.DQB1.2']] and ('3:01:00' in [row['hibag.DQA1.1'], row['hibag.DQA1.2']] or '2:01:00' in [row['hibag.DQA1.1'], row['hibag.DQA1.2']]):
        row['hibag_DQ2.2'] = 'positive'

    # Condition for hibag_DQ7.5
    if ('5:01:00' in [row['hibag.DQA1.1'], row['hibag.DQA1.2']] or '5:05:00' in [row['hibag.DQA1.1'], row['hibag.DQA1.2']]) and '3:01:00' in [row['hibag.DQB1.1'], row['hibag.DQB1.2']]:
        row['hibag_DQ7.5'] = 'positive'

    # Condition for hibag_DQ8.1
    if (('3:01:00' in [row['hibag.DQA1.1'], row['hibag.DQA1.2']] or '3:03:00' in [row['hibag.DQA1.1'], row['hibag.DQA1.2']]) and '3:02:00' in [row['hibag.DQB1.1'], row['hibag.DQB1.2']]):
        row['hibag_DQ8.1'] = 'positive'
    
    # Combining genotype risks into a single column
    genotypes = []
    if row['hibag_DQ2.5'] == 'positive':
        genotypes.append('2.5')
    if row['hibag_DQ2.2'] == 'positive':
        genotypes.append('2.2')
    if row['hibag_DQ7.5'] == 'positive':
        genotypes.append('7.5')
    if row['hibag_DQ8.1'] == 'positive':
        genotypes.append('8.1')
    
    row['hibag_genotype'] = '/'.join(genotypes) if genotypes else 'X'
    
    return row

In [ ]:
def check_combinations_hla(row):
    # Initialize the new columns
    row['hla_la_DQ2.5'], row['hla_la_DQ2.2'], row['hla_la_DQ7.5'], row['hla_la_DQ8.1'] = 'negative', 'negative', 'negative', 'negative'
    
    # Condition for hla_la_DQ2.5
    if '5:01:00' in [row['hla.la.DQA1.1'], row['hla.la.DQA1.2']] and '2:01:00' in [row['hla.la.DQB1.1'], row['hla.la.DQB1.2']]:
        row['hla_la_DQ2.5'] = 'positive'
        
    # Condition for hla_la_DQ2.2
    if '2:01:00' in [row['hla.la.DQB1.1'], row['hla.la.DQB1.2']] and '2:01:00' in [row['hla.la.DQA1.1'], row['hla.la.DQA1.2']]:
        row['hla_la_DQ2.2'] = 'positive'

    # Condition for hla_la_DQ7.5
    if ('5:01:00' in [row['hla.la.DQA1.1'], row['hla.la.DQA1.2']] or '5:05:00' in [row['hla.la.DQA1.1'], row['hla.la.DQA1.2']]) and '3:01:00' in [row['hla.la.DQB1.1'], row['hla.la.DQB1.2']]:
        row['hla_la_DQ7.5'] = 'positive'

    # Condition for hla_la_DQ8.1
    if (('3:01:00' in [row['hla.la.DQA1.1'], row['hla.la.DQA1.2']] or '3:03:00' in [row['hla.la.DQA1.1'], row['hla.la.DQA1.2']]) and '3:02:00' in [row['hla.la.DQB1.1'], row['hla.la.DQB1.2']]):
        row['hla_la_DQ8.1'] = 'positive'
    
    # Combining genotype risks into a single column
    genotypes = []
    if row['hla_la_DQ2.5'] == 'positive':
        genotypes.append('2.5')
    if row['hla_la_DQ2.2'] == 'positive':
        genotypes.append('2.2')
    if row['hla_la_DQ7.5'] == 'positive':
        genotypes.append('7.5')
    if row['hla_la_DQ8.1'] == 'positive':
        genotypes.append('8.1')
    
    row['hla_la_genotype'] = '/'.join(genotypes) if genotypes else 'X'
    
    return row

In [ ]:
HLA_concordance1 = HLA_concordance.apply(check_combinations, axis=1)

In [ ]:
HLA_concordance2 = HLA_concordance1.apply(check_combinations_hla, axis=1)

# HLA-LA

In [ ]:
def update_genotypes(row):
    genotype_map = ['2.2', '2.5', '7.5', '8.1']
    
    for geno in genotype_map:
        geno_x = f'{geno}/X'
        geno_geno = f'{geno}/{geno}'

        # Update hibag_genotype
        if row['hibag_genotype'] == geno:
            if row['DQ_geno_4'] == geno_x:
                row['hibag_genotype'] = geno_x
            elif row['DQ_geno_4'] == geno_geno:
                row['hibag_genotype'] = geno_geno

        # Update hla_la_genotype
        if row['hla_la_genotype'] == geno:
            if row['DQ_geno_prev'] == geno_x:
                row['hla_la_genotype'] = geno_x
            elif row['DQ_geno_prev'] == geno_geno:
                row['hla_la_genotype'] = geno_geno
    
    return row

In [ ]:
HLA_concordance4 = HLA_concordance2.apply(update_genotypes, axis=1)

In [ ]:
HLA_concordance4.to_csv('HLA_concordance3.csv', index=False)

In [ ]:
##manual check

In [ ]:
file_path3 = 'HLA_concordance3.csv'
HLA_concordance5 = pd.read_csv(file_path3)

## change tagSNP

In [ ]:
HLA_concordance4['tag.genotype'].unique().tolist()

In [ ]:
replacement_dict = {
    'X/X': 'X',
    'DQ2.5/X': '2.5/X',
    'DQ2.2/DQ7': '2.2/7.5',
    'DQ2.2/X': '2.2/X',
    'DQ2.5/DQ7': '2.5/7.5',
    'DQ7/X': '7.5/X',
    'DQ2.5/DQ8': '2.5/8.1',
    'DQ7/DQ7': '7.5/7.5',
    'DQ2.2/DQ8': '2.2/8.1',
    'DQ8/X': '8.1/X',
    'DQ2.5/DQ2.2': '2.5/2.2',
    'DQ8/DQ8': '8.1/8.1',
    'DQ2.5/DQ2.5': '2.5/2.5',
    'DQ2.2/DQ2.2': '2.2/2.2'
}

In [ ]:
HLA_concordance5['tag.genotype'] = HLA_concordance5['tag.genotype'].replace(replacement_dict, regex=True)

In [ ]:
HLA_concordance5.to_csv('HLA_concordance4.csv', index=False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
import seaborn as sns

In [ ]:
HLA_concordance5['tag.genotype']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
from matplotlib import rcParams

# Load the data
file_path4 = 'HLA_concordance4.csv'
HLA_concordance5 = pd.read_csv(file_path4)

# Calculate concordance
HLA_concordance5['concordant'] = (
    (HLA_concordance5['tag.genotype'] == HLA_concordance5['hibag_genotype']) &
    (HLA_concordance5['tag.genotype'] == HLA_concordance5['hla_la_genotype'])
)

# Calculate exclusive and shared overlaps
only_tag = ((HLA_concordance5['tag.genotype'] != HLA_concordance5['hibag_genotype']) & 
            (HLA_concordance5['tag.genotype'] != HLA_concordance5['hla_la_genotype'])).sum()

only_hibag = ((HLA_concordance5['hibag_genotype'] != HLA_concordance5['tag.genotype']) & 
              (HLA_concordance5['hibag_genotype'] != HLA_concordance5['hla_la_genotype'])).sum()

only_hla_la = ((HLA_concordance5['hla_la_genotype'] != HLA_concordance5['tag.genotype']) & 
               (HLA_concordance5['hla_la_genotype'] != HLA_concordance5['hibag_genotype'])).sum()

tag_and_hibag_only = ((HLA_concordance5['tag.genotype'] == HLA_concordance5['hibag_genotype']) & 
                      (HLA_concordance5['tag.genotype'] != HLA_concordance5['hla_la_genotype'])).sum()

tag_and_hla_la_only = ((HLA_concordance5['tag.genotype'] == HLA_concordance5['hla_la_genotype']) & 
                       (HLA_concordance5['tag.genotype'] != HLA_concordance5['hibag_genotype'])).sum()

hibag_and_hla_la_only = ((HLA_concordance5['hibag_genotype'] == HLA_concordance5['hla_la_genotype']) & 
                         (HLA_concordance5['hibag_genotype'] != HLA_concordance5['tag.genotype'])).sum()

all_three = ((HLA_concordance5['tag.genotype'] == HLA_concordance5['hibag_genotype']) & 
             (HLA_concordance5['tag.genotype'] == HLA_concordance5['hla_la_genotype'])).sum()

In [ ]:
# Plot Venn diagram
plt.figure(figsize=(8, 8))
venn3(
    subsets=(
        only_tag,
        only_hibag,
        tag_and_hibag_only,
        only_hla_la,
        tag_and_hla_la_only,
        hibag_and_hla_la_only,
        all_three
    ),
    set_labels=('Tag-SNP', 'HIBAG', 'HLA-LA')
)
plt.show()


### count allele

In [ ]:
alleles_of_interest = ['2.2', '2.5', '7.5', '8.1']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
from matplotlib import rcParams

# Assume the data is already loaded into HLA_concordance5
# HLA_concordance5 = pd.read_csv('HLA_concordance4.csv') # Example path

# Define the alleles of interest
alleles_of_interest = ['2.2', '2.5', '7.5', '8.1']

# Function to get allele presence as a set for each column
def get_allele_presence(df, allele):
    tag_presence = df['tag.genotype'].apply(lambda x: allele in x.split('/'))
    hibag_presence = df['hibag_genotype'].apply(lambda x: allele in x.split('/'))
    hla_la_presence = df['hla_la_genotype'].apply(lambda x: allele in x.split('/'))
    return tag_presence, hibag_presence, hla_la_presence

# Set the font properties to Arial and size 16
rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 16

# Create a 2x2 subplot
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

# Plot Venn diagrams for each allele of interest
for i, allele in enumerate(alleles_of_interest):
    tag_presence, hibag_presence, hla_la_presence = get_allele_presence(HLA_concordance5, allele)
    
    # Calculate specific overlaps
    only_tag = ((tag_presence) & (~hibag_presence) & (~hla_la_presence)).sum()
    only_hibag = ((~tag_presence) & (hibag_presence) & (~hla_la_presence)).sum()
    only_hla_la = ((~tag_presence) & (~hibag_presence) & (hla_la_presence)).sum()
    tag_and_hibag_only = ((tag_presence) & (hibag_presence) & (~hla_la_presence)).sum()
    tag_and_hla_la_only = ((tag_presence) & (~hibag_presence) & (hla_la_presence)).sum()
    hibag_and_hla_la_only = ((~tag_presence) & (hibag_presence) & (hla_la_presence)).sum()
    all_three = ((tag_presence) & (hibag_presence) & (hla_la_presence)).sum()
    
    # Plot Venn diagram
    ax = axes[i]
    venn = venn3(
        subsets=(
            only_tag,
            only_hibag,
            tag_and_hibag_only,
            only_hla_la,
            tag_and_hla_la_only,
            hibag_and_hla_la_only,
            all_three
        ),
        set_labels=('Tag-SNP', 'HIBAG', 'HLA-LA'),
        ax=ax
    )
    
    # Adjust font sizes
    for label in venn.set_labels:
        label.set_fontsize(16)
    for label in venn.subset_labels:
        if label:
            label.set_fontsize(16)

# Adjust layout and remove titles
plt.tight_layout()
for ax in axes:
    ax.set_title('')

# Save the figure to a PDF
output_pdf_path = 'allele_concordance_venn_diagrams.pdf'
plt.savefig(output_pdf_path, format='pdf')

plt.show()
